In [1]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

In [2]:
mp_pose = mp.solutions.pose
 
pose_image = mp_pose.Pose(static_image_mode=True, 
                          min_detection_confidence=0.5)
 
pose_video = mp_pose.Pose(static_image_mode=False, 
                          min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)
 
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose()

In [3]:
def detectPose(image_pose, pose, draw=True, display=False):
    original_image = image_pose.copy()
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
    resultant = pose.process(image_in_RGB)
    if resultant.pose_landmarks and draw:    
        mp_drawing.draw_landmarks(image=original_image, 
                                  landmark_list=resultant.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                               thickness=2, circle_radius=2),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                               thickness=2, circle_radius=2))
    if display:
            plt.figure(figsize=[22,22])
            plt.subplot(121);plt.imshow(image_pose[:,:,::-1])
            plt.title("Input Image",size=14)
            plt.axis('off');
            plt.subplot(122);plt.imshow(original_image[:,:,::-1])
            plt.title("Pose detected Image",size=14)
            plt.axis('off');
 
    else:        
        return original_image, resultant


In [4]:
# PREPARING DATA - DICTIONARY WITH POINTS ON ALL AXIS

image = cv2.imread("20230123_182353.jpg")
slika, res = detectPose(image, pose)

points_data = {}
points_x = []
points_y = []
points_z = []
points_visible = []

for point in res.pose_landmarks.landmark:
    points_x.append(point.x)
    points_y.append(point.y)
    points_z.append(point.z)
    points_visible.append(point.visibility)

points_data["x"] = points_x
points_data["y"] = points_y
points_data["z"] = points_z
points_data["visibility"] = points_visible

